In [1]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/asaezco/src/devel2/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))
    sys.path.append("/var/SP/data/home/asaezco/src/devel2/pykhaos")
    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

Added '/var/SP/data/home/asaezco/src/devel2/use-cases' to path
Added '/var/SP/data/home/asaezco/src/devel2' to path


In [2]:
# coding=utf-8

import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from utils_model import *
from utils_fbb_churn import *
from metadata_fbb_churn import *
from feature_selection_utils import *
import subprocess


In [3]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

In [4]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.low"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS

In [5]:
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

In [6]:
def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [7]:
spark = initialize("FBB Improvement ",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 23.902078867 secs | default parallelism=24


In [8]:
def getFbbChurnLabeledCarCycles_port(spark, origin, yearmonthday, selcols, horizon = 4):

    cycle = 0
    fini_tmp = yearmonthday
    while cycle < horizon:
        yearmonthday_target = get_next_cycle(fini_tmp, str_fmt="%Y%m%d")
        cycle = cycle + 1
        fini_tmp = yearmonthday_target

    yearmonth = yearmonthday[0:6]

    trfeatdf = getCarNumClienteDf(spark, origin, yearmonthday)

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Samples for month " + yearmonthday + ": " + str(trfeatdf.count()))

    # Loading port-out requests and DXs
    # # labmonthlisttr = getMonthSeq(initportmonthtr, lastportmonthtr)

    # Las bajas de fibra pueden venir por:
    #- Solicitudes de baja de fijo
    fixporttr = getFixPortRequestsForCycleList(spark, yearmonthday, yearmonthday_target)
    #- Porque dejen de estar en la lista de clientes
    fixdxtr = getFbbDxsForCycleList(spark,yearmonthday, yearmonthday_target)

    # Labeling: FBB service is labeled as 1 if, during the next time window specified by the horizon, either the associated fixed service requested to be ported out or the FBB was disconnected
    window = Window.partitionBy("num_cliente")

    unbaltrdf = trfeatdf\
    .join(fixporttr, ['msisdn_d'], "left_outer")\
    .na.fill({'label_srv': 0.0})\
    .join(fixdxtr, ['msisdn'], "left_outer")\
    .na.fill({'label_dx': 0.0})\
    .withColumn('tmp', when((col('label_srv')==1.0), 1.0).otherwise(0.0))\
    .withColumn('label', sql_max('tmp').over(window))\
    .filter(col("rgu")=="fbb")\
    .select(selcols + ['label'])

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Labeled samples for month " + yearmonth + ": " + str(unbaltrdf.count()))

    return unbaltrdf

In [9]:
def getFbbChurnLabeledCarCycles_dx(spark, origin, yearmonthday, selcols, horizon = 4):

    cycle = 0
    fini_tmp = yearmonthday
    while cycle < horizon:
        yearmonthday_target = get_next_cycle(fini_tmp, str_fmt="%Y%m%d")
        cycle = cycle + 1
        fini_tmp = yearmonthday_target

    yearmonth = yearmonthday[0:6]

    trfeatdf = getCarNumClienteDf(spark, origin, yearmonthday)

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Samples for month " + yearmonthday + ": " + str(trfeatdf.count()))

    # Loading port-out requests and DXs
    # # labmonthlisttr = getMonthSeq(initportmonthtr, lastportmonthtr)

    # Las bajas de fibra pueden venir por:
    #- Solicitudes de baja de fijo
    fixporttr = getFixPortRequestsForCycleList(spark, yearmonthday, yearmonthday_target)
    #- Porque dejen de estar en la lista de clientes
    fixdxtr = getFbbDxsForCycleList(spark,yearmonthday, yearmonthday_target)

    # Labeling: FBB service is labeled as 1 if, during the next time window specified by the horizon, either the associated fixed service requested to be ported out or the FBB was disconnected
    window = Window.partitionBy("num_cliente")

    unbaltrdf = trfeatdf\
    .join(fixporttr, ['msisdn_d'], "left_outer")\
    .na.fill({'label_srv': 0.0})\
    .join(fixdxtr, ['msisdn'], "left_outer")\
    .na.fill({'label_dx': 0.0})\
    .withColumn('tmp', when((col('label_dx')==1.0), 1.0).otherwise(0.0))\
    .withColumn('label', sql_max('tmp').over(window))\
    .filter(col("rgu")=="fbb")\
    .select(selcols + ['label'])

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Labeled samples for month " + yearmonth + ": " + str(unbaltrdf.count()))

    return unbaltrdf

In [18]:
def getFbbChurnLabeledCarCycles_both(spark, origin, yearmonthday, selcols, horizon = 4):

    cycle = 0
    fini_tmp = yearmonthday
    while cycle < horizon:
        yearmonthday_target = get_next_cycle(fini_tmp, str_fmt="%Y%m%d")
        cycle = cycle + 1
        fini_tmp = yearmonthday_target

    yearmonth = yearmonthday[0:6]

    trfeatdf = getCarNumClienteDf(spark, origin, yearmonthday)

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Samples for month " + yearmonthday + ": " + str(trfeatdf.count()))

    # Loading port-out requests and DXs
    # # labmonthlisttr = getMonthSeq(initportmonthtr, lastportmonthtr)

    # Las bajas de fibra pueden venir por:
    #- Solicitudes de baja de fijo
    fixporttr = getFixPortRequestsForCycleList(spark, yearmonthday, yearmonthday_target)
    #- Porque dejen de estar en la lista de clientes
    fixdxtr = getFbbDxsForCycleList(spark,yearmonthday, yearmonthday_target)

    # Labeling: FBB service is labeled as 1 if, during the next time window specified by the horizon, either the associated fixed service requested to be ported out or the FBB was disconnected
    window = Window.partitionBy("num_cliente")

    unbaltrdf = trfeatdf\
    .join(fixporttr, ['msisdn_d'], "left_outer")\
    .na.fill({'label_srv': 0.0})\
    .join(fixdxtr, ['msisdn'], "left_outer")\
    .na.fill({'label_dx': 0.0})\
    .withColumn('tmp', when((col('label_dx')==1.0), 1.0).otherwise(0.0))\
    .withColumn('label_bajas', sql_max('tmp').over(window))\
    .withColumn('tmp2', when((col('label_srv')==1.0), 1.0).otherwise(0.0))\
    .withColumn('label_port', sql_max('tmp2').over(window))\
    .filter(col("rgu")=="fbb")\
    .select(selcols + ['label_port', 'label_bajas'])

    print("[Info getFbbChurnLabeledCar] " + time.ctime() + " Labeled samples for month " + yearmonth + ": " + str(unbaltrdf.count()))

    return unbaltrdf

In [19]:
set_paths()

from pykhaos.utils.date_functions import *
from utils_fbb_churn import *
global sqlContext

In [20]:
selcols = getIdFeats() + getCrmFeats() + getBillingFeats() + getMobSopoFeats() + getOrdersFeats()
now = datetime.now()

date_name = str(now.year) + str(now.month).rjust(2, '0') + str(now.day).rjust(2, '0')
origin = '/user/hive/warehouse/tests_es.db/jvmm_amdocs_ids_'

In [21]:
trcycle_ini = '20181130'  # Training data
    # Number of cycles to gather dismiss requests
horizon = 4
    # Cycle used for CAR and Extra Feats in the test set
ttcycle_ini = '20181231'  # Test data

tr_ttdates = trcycle_ini + '_' + ttcycle_ini

In [22]:
inittrdf_ini_port = getFbbChurnLabeledCarCycles_port(spark, origin, trcycle_ini, selcols, horizon)
inittrdf_ini_dx= getFbbChurnLabeledCarCycles_dx(spark, origin, trcycle_ini, selcols, horizon)

KeyboardInterrupt: 

In [23]:
inittrdf_ini = getFbbChurnLabeledCarCycles_both(spark, origin, trcycle_ini, selcols, horizon)

[Info getCarNumClienteDf] Tue Jul 23 08:31:41 2019 Size of the original df for 20181130: 13248445
[Info FbbChurn] Tue Jul 23 08:35:11 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Tue Jul 23 08:35:12 2019 Size of the origindf after load from HDFS: 13248445
[Info getCarNumClienteDf] Tue Jul 23 08:36:15 2019 Size of the numclidf for 20181130 - Num rows: 13248445 - Num columns: 101
[Info FbbChurn] Tue Jul 23 08:38:51 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Tue Jul 23 08:38:51 2019 Starting the preparation of billing feats
[Info get_billing_df] Tue Jul 23 08:38:55 2019 Size of the df with billing feats: 5714359
[Info getCarNumClienteDf] Tue Jul 23 08:38:58 2019 Size of the tmpdf for 20181130 - Num rows: 13248445 - Num columns: 134
[Info get_mobile_spinners_df] Tue Jul 23 08:39:01 2019 Port-out data loaded for 20181130 with a total of 27468495 rows and 11448545 distinct NIFs
[Info get_mobile_spinners_df] Tue Jul 23 08:39:03 2019 Timing feats computed for a total 

In [24]:
inittrdf_ini.printSchema()

root
 |-- rgu: string (nullable = true)
 |-- campo1: string (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- num_cliente: string (nullable = true)
 |-- msisdn_d: string (nullable = true)
 |-- nif_cliente: string (nullable = true)
 |-- mean_dias_desde_fx_srv_basic: double (nullable = false)
 |-- movil_services: long (nullable = true)
 |-- tv_services: long (nullable = true)
 |-- total_penal_cust_pending_n3_penal_amount: double (nullable = false)
 |-- x_formato_factura: string (nullable = true)
 |-- x_user_twitter: string (nullable = true)
 |-- total_penal_cust_pending_n4_penal_amount: double (nullable = false)
 |-- dias_desde_bam-movil_fx_first: double (nullable = false)
 |-- max_dias_desde_fx_trybuy_autom_tv: double (nullable = false)
 |-- total_price_tariff: double (nullable = false)
 |-- flg_robinson: string (nullable = true)
 |-- pvr_services: double (nullable = false)
 |-- num_tariff_redl: double (nullable = false)
 |-- dias_desde_fx_fbb_upgrade: double (nullable = fals

In [29]:
inittrdf_ini.where(col('label_bajas') > 0).count()

63349

In [30]:
inittrdf_ini.where(col('label_port') > 0).count()

21643

In [ ]:
save_dir = save_dir = 'tests_es.asaezco_fbb_improvement_dx'
inittrdf_ini_dx.write.format('parquet').mode('overwrite').saveAsTable(save_dir)

In [16]:
save_dir = save_dir = 'tests_es.asaezco_fbbb_improvement_port'
inittrdf_ini_port.write.format('parquet').mode('overwrite').saveAsTable(save_dir)

In [20]:
inittrdf_ini_dx = spark.read.parquet('hdfs://nameservice1/user/hive/warehouse/tests_es.db/asaezco_fbbb_improvement_dx')

In [27]:
## Reading the Extra Features
dfExtraFeat = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                         .format(int(trcycle_ini[0:4]), int(trcycle_ini[4:6]), int(trcycle_ini[6:8])))

    # Taking only the clients with a fbb service
dfExtraFeatfbb = dfExtraFeat.join(inittrdf_ini, ["num_cliente"], "leftsemi")

dfExtraFeatfbb = dfExtraFeatfbb.cache()
#print "[Info Main FbbChurn] " + time.ctime() + " Count of the ExtraFeats: ", dfExtraFeatfbb.count()

    # Taking the Extra Features of interest and adding their values for num_client when necessary
dfExtraFeatSel, selColumnas = addExtraFeatsEvol(dfExtraFeatfbb)

#print "[Info Main FbbChurn] " + time.ctime() + " Calculating the total value of the extra feats for each number client"

dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeatSel.columns:
        dfillNa.pop(kkey, None)

[Info FbbChurn] Tue Jul 23 08:45:21 2019 Inside CCC module of the Extra Feats 
[Info FbbChurn] Tue Jul 23 08:45:22 2019 Number of columns processed: 205
[Info FbbChurn] Number of rows: 8335417
[Info FbbChurn] Tue Jul 23 08:48:51 2019 The module for CCC has been run. Number of clients  1978186
[Info FbbChurn] Tue Jul 23 08:49:06 2019 Inside NS module of the Extra Feats 
[Info FbbChurn] Tue Jul 23 08:49:07 2019 Number of columns processed: 608
[Info FbbChurn] Tue Jul 23 08:49:14 2019 The module for NS has been run. Number of clients  1978186
[Info FbbChurn] Tue Jul 23 08:50:32 2019 Inside TGS module of the Extra Feats 
[Info FbbChurn] Tue Jul 23 08:50:32 2019 Number of columns processed: 21
[Info FbbChurn] Tue Jul 23 08:50:33 2019 The module for TGS has been run. Number of clientes  1978186
[Info FbbChurn] Tue Jul 23 08:50:35 2019 Inside Orders module of the Extra Feats 
[Info FbbChurn] Tue Jul 23 08:50:35 2019 Number of columns processed: 8
[Info FbbChurn] Tue Jul 23 08:50:36 2019 The m

In [ ]:
inittrdf = inittrdf_ini.join(dfExtraFeatSel, ["msisdn", "num_cliente", 'rgu'], how="left").na.fill(dfillNa)

In [31]:
[unbaltrdf, valdf] = inittrdf.randomSplit([0.7, 0.3], 1234)

In [32]:
unbaltrdf = unbaltrdf.cache()
valdf = valdf.cache()

In [34]:
unbaltrdf.groupBy('label_bajas').agg(count('*')).show()
unbaltrdf.groupBy('label_port').agg(count('*')).show()

trdf_port = balance_df2(unbaltrdf, 'label_port')
trdf_dx = balance_df2(unbaltrdf, 'label_bajas')

+-----------+--------+
|label_bajas|count(1)|
+-----------+--------+
|        0.0| 1341592|
|        1.0|   44279|
+-----------+--------+

+----------+--------+
|label_port|count(1)|
+----------+--------+
|       0.0| 1370718|
|       1.0|   15153|
+----------+--------+

[Info balance_df2] Dataframe has been balanced - Total number of rows is 30339
[Info balance_df2] Dataframe has been balanced - Total number of rows is 88358


In [35]:
trdf_dx.groupBy('label_bajas').agg(count('*')).show()
trdf_port.groupBy('label_port').agg(count('*')).show()

+-----------+--------+
|label_bajas|count(1)|
+-----------+--------+
|        0.0|   44079|
|        1.0|   44279|
+-----------+--------+

+----------+--------+
|label_port|count(1)|
+----------+--------+
|       0.0|   15186|
|       1.0|   15153|
+----------+--------+



In [36]:
allFeats = trdf_dx.columns

# Getting only the numeric variables
catCols = [item[0] for item in trdf_dx.dtypes if item[1].startswith('string')]
numerical_feats = list(set(allFeats) - set(list(
        set().union(getIdFeats(), getIdFeats_tr(), getNoInputFeats(), catCols, [c + "_enc" for c in getCatFeatsCrm()],
                    ["label"]))))

noninf_feats = getNonInfFeats(trdf_dx, numerical_feats)

[Info FbbChurn] 1/8 Selecting the features that add information to the model
[Info FbbChurn] 2/8 Selecting the features that add information to the model
[Info FbbChurn] 3/8 Selecting the features that add information to the model
[Info FbbChurn] 4/8 Selecting the features that add information to the model
[Info FbbChurn] 5/8 Selecting the features that add information to the model
[Info FbbChurn] 6/8 Selecting the features that add information to the model
[Info FbbChurn] 7/8 Selecting the features that add information to the model
[Info FbbChurn] 8/8 Selecting the features that add information to the model


# Test

In [37]:
ttdf_ini = getFbbChurnLabeledCarCycles_both(spark, origin, ttcycle_ini, selcols,horizon)

[Info getCarNumClienteDf] Tue Jul 23 09:29:45 2019 Size of the original df for 20181231: 13683905
[Info FbbChurn] Tue Jul 23 09:32:44 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Tue Jul 23 09:32:45 2019 Size of the origindf after load from HDFS: 13683905
[Info getCarNumClienteDf] Tue Jul 23 09:34:07 2019 Size of the numclidf for 20181231 - Num rows: 13683905 - Num columns: 101
[Info FbbChurn] Tue Jul 23 09:37:01 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Tue Jul 23 09:37:02 2019 Starting the preparation of billing feats
[Info get_billing_df] Tue Jul 23 09:37:15 2019 Size of the df with billing feats: 5825804
[Info getCarNumClienteDf] Tue Jul 23 09:37:18 2019 Size of the tmpdf for 20181231 - Num rows: 13683905 - Num columns: 134
[Info get_mobile_spinners_df] Tue Jul 23 09:37:26 2019 Port-out data loaded for 20181231 with a total of 27844643 rows and 11499038 distinct NIFs
[Info get_mobile_spinners_df] Tue Jul 23 09:37:40 2019 Timing feats computed for a total 

In [38]:
dfExtraFeat_tt = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                            .format(int(ttcycle_ini[0:4]), int(ttcycle_ini[4:6]), int(ttcycle_ini[6:8])))
dfExtraFeatfbb_tt = dfExtraFeat_tt.join(ttdf_ini.select('num_cliente'), on='num_cliente', how='leftsemi')

In [39]:
dfExtraFeatfbb_tt = dfExtraFeatfbb_tt.cache()

In [40]:
dfExtraFeat_ttSel, selColumnas = addExtraFeatsEvol(dfExtraFeatfbb_tt)

[Info FbbChurn] Tue Jul 23 09:41:39 2019 Inside CCC module of the Extra Feats 
[Info FbbChurn] Tue Jul 23 09:41:39 2019 Number of columns processed: 205
[Info FbbChurn] Number of rows: 8701397
[Info FbbChurn] Tue Jul 23 09:45:02 2019 The module for CCC has been run. Number of clients  2050838
[Info FbbChurn] Tue Jul 23 09:45:16 2019 Inside NS module of the Extra Feats 
[Info FbbChurn] Tue Jul 23 09:45:18 2019 Number of columns processed: 608
[Info FbbChurn] Tue Jul 23 09:45:25 2019 The module for NS has been run. Number of clients  2050838
[Info FbbChurn] Tue Jul 23 09:46:47 2019 Inside TGS module of the Extra Feats 
[Info FbbChurn] Tue Jul 23 09:46:48 2019 Number of columns processed: 21
[Info FbbChurn] Tue Jul 23 09:46:48 2019 The module for TGS has been run. Number of clientes  2050838
[Info FbbChurn] Tue Jul 23 09:46:51 2019 Inside Orders module of the Extra Feats 
[Info FbbChurn] Tue Jul 23 09:46:51 2019 Number of columns processed: 8
[Info FbbChurn] Tue Jul 23 09:46:51 2019 The m

In [41]:
dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeat_ttSel.columns:
        dfillNa.pop(kkey, None)

ttdf = ttdf_ini.join(dfExtraFeat_ttSel, ["msisdn", "num_cliente", 'rgu'], how="left").na.fill(dfillNa)

In [68]:
featCols = list(set(numerical_feats) - set(noninf_feats + ['label_bajas','label_port']))

In [71]:
assembler = VectorAssembler(inputCols=featCols, outputCol="features")

classifier_port = RandomForestClassifier(featuresCol="features", \
                                        labelCol="label_port", \
                                        maxDepth=20, \
                                        maxBins=32, \
                                        minInstancesPerNode=100, \
                                        impurity="entropy", \
                                        featureSubsetStrategy="sqrt", \
                                        subsamplingRate=0.85, minInfoGain = 0.001, \
                                        numTrees=800, \
                                        seed=1234)  
classifier_dx = RandomForestClassifier(featuresCol="features", \
                                        labelCol="label_bajas", \
                                        maxDepth=20, \
                                        maxBins=32, \
                                        minInstancesPerNode=100, \
                                        impurity="entropy", \
                                        featureSubsetStrategy="sqrt", \
                                        subsamplingRate=0.85, minInfoGain = 0.001, \
                                        numTrees=800, \
                                        seed=1234)     

pipeline_port = Pipeline(stages=[assembler, classifier_port])
pipeline_dx = Pipeline(stages=[assembler, classifier_dx])
    
getScore = udf(lambda prob: float(prob[1]), DoubleType())

In [72]:
model_dx = pipeline_dx.fit(trdf_dx)
calibmodel_dx = getCalibrationFunction(spark, model_dx, valdf, 'label_bajas', 10)

[Info Calibration] freqprob: key = 0.0 - value = 0.00247142415817
[Info Calibration] freqprob: key = 1.0 - value = 0.00483859975921
[Info Calibration] freqprob: key = 2.0 - value = 0.0115453054189
[Info Calibration] freqprob: key = 3.0 - value = 0.0203009876156
[Info Calibration] freqprob: key = 4.0 - value = 0.0271321205978
[Info Calibration] freqprob: key = 5.0 - value = 0.0416190570818
[Info Calibration] freqprob: key = 6.0 - value = 0.0675408877081
[Info Calibration] freqprob: key = 7.0 - value = 0.128364286424
[Info Calibration] freqprob: key = 8.0 - value = 0.264437242192
[Info Calibration] freqprob: key = 9.0 - value = 0.472338204593
[Info Calibration] midpoints: key = 0.0 - value = 0.13543528697484117
[Info Calibration] midpoints: key = 1.0 - value = 0.22058367182820016
[Info Calibration] midpoints: key = 2.0 - value = 0.3057320566815592
[Info Calibration] midpoints: key = 3.0 - value = 0.3908804415349182
[Info Calibration] midpoints: key = 4.0 - value = 0.47602882638827715
[In

In [78]:
model_port = pipeline_dx.fit(trdf_port)
calibmodel_port = getCalibrationFunction(spark, model_port, valdf, 'label_port', 10)

[Info Calibration] freqprob: key = 0.0 - value = 0.00028133917447
[Info Calibration] freqprob: key = 1.0 - value = 0.0011390353247
[Info Calibration] freqprob: key = 2.0 - value = 0.00309442790302
[Info Calibration] freqprob: key = 3.0 - value = 0.00714274905041
[Info Calibration] freqprob: key = 4.0 - value = 0.0113981411551
[Info Calibration] freqprob: key = 5.0 - value = 0.0178279247341
[Info Calibration] freqprob: key = 6.0 - value = 0.0310793340896
[Info Calibration] freqprob: key = 7.0 - value = 0.0504482709549
[Info Calibration] freqprob: key = 8.0 - value = 0.0753571428571
[Info Calibration] freqprob: key = 9.0 - value = 0.104347826087
[Info Calibration] midpoints: key = 0.0 - value = 0.09849312126199095
[Info Calibration] midpoints: key = 1.0 - value = 0.16484806403329175
[Info Calibration] midpoints: key = 2.0 - value = 0.23120300680459255
[Info Calibration] midpoints: key = 3.0 - value = 0.2975579495758933
[Info Calibration] midpoints: key = 4.0 - value = 0.36391289234719415

In [73]:
tr_preds_df_dx = model_dx.transform(trdf_dx).withColumn("model_score", getScore(col("probability")).cast(DoubleType()))
tr_calib_preds_df_dx = calibmodel_dx[0].transform(tr_preds_df_dx)

In [74]:
trPredictionAndLabels_dx = tr_calib_preds_df_dx.select(['calib_model_score', 'label_bajas']).rdd.map(lambda r: (r['calib_model_score'], r['label_bajas']))
trmetrics_dx = BinaryClassificationMetrics(trPredictionAndLabels_dx)

In [75]:
tt_preds_df_dx = model_dx.transform(ttdf).withColumn("model_score", getScore(col("probability")).cast(DoubleType()))
tt_calib_preds_df_dx = calibmodel_dx[0].transform(tt_preds_df_dx)

In [76]:
ttPredictionAndLabels_dx = tt_calib_preds_df_dx.select(['calib_model_score', 'label_bajas']).rdd.map(lambda r: (r['calib_model_score'], r['label_bajas']))
ttmetrics_dx = BinaryClassificationMetrics(ttPredictionAndLabels_dx)

In [77]:
print('Bajas:')
print(" Area under ROC(tr) = " + str(trmetrics_dx.areaUnderROC))
print(" Area under ROC(tt) = " + str(ttmetrics_dx.areaUnderROC))    
print(" ")

Bajas:
 Area under ROC(tr) = 0.785218095848
 Area under ROC(tt) = 0.715325882966
 


In [85]:
feat_imp = model_dx.stages[1].featureImportances.toArray()

In [86]:
idx = np.argsort(feat_imp)[::-1]

In [87]:
feat_array = np.asarray(featCols)

In [98]:
inf = 0
sup = 10
idxs = idx[inf:sup]
imps = feat_imp[idxs]

In [97]:
feat_array[idxs]

array(['ccc_most_common_bucket_interactions_total',
       'ccc_num_diff_buckets_total',
       'ccc_days_since_first_interaction_total',
       'ccc_bucket_churn_cancellations_total',
       'ccc_days_since_latest_interaction_total',
       'ccc_num_interactions_total', 'dias_desde_fixed_fx_first',
       'ccc_raw_resultado_no_retenido_total', 'dias_desde_fbb_fx_first',
       'min_dias_desde_fx_srv_basic'], dtype='|S96')

In [79]:
tr_preds_df_port = model_port.transform(trdf_port).withColumn("model_score", getScore(col("probability")).cast(DoubleType()))
tr_calib_preds_df_port = calibmodel_port[0].transform(tr_preds_df_port)

In [80]:
trPredictionAndLabels_port = tr_calib_preds_df_port.select(['calib_model_score', 'label_port']).rdd.map(lambda r: (r['calib_model_score'], r['label_port']))
trmetrics_port = BinaryClassificationMetrics(trPredictionAndLabels_port)

In [82]:
tt_preds_df_port = model_port.transform(ttdf).withColumn("model_score", getScore(col("probability")).cast(DoubleType()))
tt_calib_preds_df_port = calibmodel_port[0].transform(tt_preds_df_port)

In [83]:
ttPredictionAndLabels_port = tt_calib_preds_df_port.select(['calib_model_score', 'label_port']).rdd.map(lambda r: (r['calib_model_score'], r['label_port']))
ttmetrics_port = BinaryClassificationMetrics(ttPredictionAndLabels_port)

In [84]:
print('Portas:')
print(" Area under ROC(tr) = " + str(trmetrics_port.areaUnderROC))
print(" Area under ROC(tt) = " + str(ttmetrics_port.areaUnderROC))    
print(" ")

Portas:
 Area under ROC(tr) = 0.779710719918
 Area under ROC(tt) = 0.725350916552
 


In [116]:
feat_imp_p = model_port.stages[1].featureImportances.toArray()

In [117]:
idx_p = np.argsort(feat_imp_p)[::-1]

In [118]:
feat_array = np.asarray(featCols)

In [119]:
inf = 0
sup = 10
idxs_p = idx_p[inf:sup]

In [104]:
feats_bajas = zip(feat_array[idxs], imps)

In [126]:
feats_portas = zip(feat_array[idx_p], feat_imp_p[idx_p])

In [127]:
feats_bajas

[('ccc_most_common_bucket_interactions_total', 0.027116637857231958),
 ('ccc_num_diff_buckets_total', 0.025255509020595594),
 ('ccc_days_since_first_interaction_total', 0.023181305886960848),
 ('ccc_bucket_churn_cancellations_total', 0.02303317151027158),
 ('ccc_days_since_latest_interaction_total', 0.02293490083581136),
 ('ccc_num_interactions_total', 0.02217748694542236),
 ('dias_desde_fixed_fx_first', 0.015588031371729632),
 ('ccc_raw_resultado_no_retenido_total', 0.015312672679620656),
 ('dias_desde_fbb_fx_first', 0.015178777674648442),
 ('min_dias_desde_fx_srv_basic', 0.014482427023327032)]

In [128]:
feats_portas

[('tgs_days_until_f_fin_bi', 0.04437625038356729),
 ('tgs_days_until_fecha_fin_dto', 0.030981844315489462),
 ('Bill_N1_Tax_Amount', 0.02277415508555941),
 ('Bill_N1_InvoiceCharges', 0.020294511257730193),
 ('Bill_N1_Amount_To_Pay', 0.020287174730944664),
 ('bill_n1_net', 0.019352215803931167),
 ('ccc_num_diff_buckets_total', 0.017610054753589083),
 ('ccc_num_interactions_total', 0.017033302364966765),
 ('tgs_days_since_f_inicio_bi', 0.01694593152408463),
 ('ccc_most_common_bucket_interactions_total', 0.01672597922811537),
 ('ccc_bucket_churn_cancellations_total', 0.014570574078764472),
 ('ccc_days_since_first_interaction_total', 0.01340039967702006),
 ('ccc_days_since_latest_interaction_total', 0.011847476503985582),
 ('dias_desde_fixed_fx_first', 0.01173086190283765),
 ('total_penal_srv_pending_n1_penal_amount', 0.011350227309251516),
 ('ccc_num_na_buckets_total', 0.009678676579828988),
 ('dias_desde_movil_fx_first', 0.009555297065706516),
 ('max_dias_desde_fx_srv_basic', 0.0092352636

0

Model 0: Area under ROC(tr) = 0.754406933206

Model 0: Area under ROC(tt) = 0.713514753724

1

Model 1: Area under ROC(tr) = 0.755106476011

Model 1: Area under ROC(tt) = 0.713128028314

2

Model 2: Area under ROC(tr) = 0.753097394074

Model 2: Area under ROC(tt) = 0.712410491476

3

Model 3: Area under ROC(tr) = 0.755560008586

Model 3: Area under ROC(tt) = 0.712465860634

4

Model 4: Area under ROC(tr) = 0.755012309843

Model 4: Area under ROC(tt) = 0.713019849073

In [ ]:
spark.stop()